In [13]:
from pyDOE import *
import pygmo as pg

In [21]:
solutions = lhs(3, samples=1000)
solutions

array([[0.16241871, 0.60838   , 0.1173387 ],
       [0.1454952 , 0.73986369, 0.19873526],
       [0.76884192, 0.69083683, 0.5236719 ],
       ...,
       [0.22930426, 0.31049665, 0.94537948],
       [0.75996577, 0.54440067, 0.40485766],
       [0.11417734, 0.22778582, 0.59744035]])

In [15]:
prob = pg.problem(dtlz(prob_id = 1, dim = 12, fdim = 3, alpha = 100))

In [17]:
prob.fitness(solutions[0])

ValueError: cannot convert a NumPy array to a C++ vector: the array must be unidimensional, but the dimension is 2 instead

In [18]:
import os
#import required modules.

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [22]:
train = solutions

In [64]:
trn_x_torch = torch.from_numpy(train)
trn_dataloader = torch.utils.data.DataLoader(trn_x_torch,batch_size=100,shuffle=False, num_workers=4)
trn_x_torch

tensor([[0.1624, 0.6084, 0.1173],
        [0.1455, 0.7399, 0.1987],
        [0.7688, 0.6908, 0.5237],
        ...,
        [0.2293, 0.3105, 0.9454],
        [0.7600, 0.5444, 0.4049],
        [0.1142, 0.2278, 0.5974]], dtype=torch.float64)

In [84]:
class AutoEncoder(nn.Module):
    
    def __init__(self):
        super(AutoEncoder, self).__init__()
        
        #encoder
        self.e1 = nn.Linear(3,24)
        self.e2 = nn.Linear(24,256)
        self.e3 = nn.Linear(256,128)
        '''
        #Latent View
        self.lv = nn.Linear(256,12)
        
        #Decoder
        self.d1 = nn.Linear(12,64)
        self.d2 = nn.Linear(64,128)
        '''
        self.output_layer = nn.Linear(128,12)
        
    def forward(self,x):
        x = F.relu(self.e1(x.float()))
        x = F.relu(self.e2(x))
        x = F.relu(self.e3(x))
        '''
        x = torch.sigmoid(self.lv(x))
        
        x = F.relu(self.d1(x))
        x = F.relu(self.d2(x))
        '''
        x = self.output_layer(x)
        x = torch.from_numpy(prob.fitness(x.detach().numpy()))
        return x

In [85]:
ae = AutoEncoder()
print(ae)

AutoEncoder(
  (e1): Linear(in_features=3, out_features=24, bias=True)
  (e2): Linear(in_features=24, out_features=256, bias=True)
  (e3): Linear(in_features=256, out_features=128, bias=True)
  (output_layer): Linear(in_features=128, out_features=12, bias=True)
)


In [86]:
#define our optimizer and loss function
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(ae.parameters(), lr=1e-3)

In [87]:
losses = []
EPOCHS = 5
for epoch in range(EPOCHS):
    for data in trn_dataloader:
        for vector in data:
            vector = torch.autograd.Variable(vector)
            optimizer.zero_grad()

            #sol_pred = ae(vector)
            pred = ae(vector)
            
            loss = loss_func(pred, vector)

            losses.append(loss.cpu().data.item())
            # Backpropagation
            loss.backward()

            optimizer.step()


            # Display
            if batch_idx % 100 == 1:
                print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch+1,
                    EPOCHS,
                    batch_idx * len(data), 
                    len(trn_dataloader.dataset),
                    100. * batch_idx / len(trn_dataloader), 
                    loss.cpu().data.item()), 
                    end='')

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
losses = []
EPOCHS = 5
for epoch in range(EPOCHS):
    
    for batch_idx, (data,target) in enumerate(trn_dataloader):
        
        data = torch.autograd.Variable(data)
    
        
        optimizer.zero_grad()
        
        pred = ae(data)
        true = 
        loss = loss_func(pred, data)
        
        losses.append(loss.cpu().data.item())
        
        # Backpropagation
        loss.backward()
        
        optimizer.step()
        
        
        # Display
        if batch_idx % 100 == 1:
            print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1,
                EPOCHS,
                batch_idx * len(data), 
                len(trn_dataloader.dataset),
                100. * batch_idx / len(trn_dataloader), 
                loss.cpu().data.item()), 
                end='')
